# Project: Wrangling and Analyze Data

In [58]:
import sys
sys.executable

'C:\\Users\\Nadav\\Documents\\ALL_PROJECTS\\dog_data_wrangling\\venv\\Scripts\\python.exe'

## Data Gathering

In [59]:
import pandas as pd

In [60]:
import gather # gathering functions module, shift+tab on each function call bellow for more details.

1. ~Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)~

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

`gather.download_file_from_url(url_path='link.txt', destination='image_predictions.tsv')` **\*file already exists**

In [61]:
help(gather.download_file_from_url) # doc for the above function

Help on function download_file_from_url in module gather:

download_file_from_url(url_path, destination)
    Save File From a given url.
    
    :param url_path: (str) a path a file containing the url.
    :param destination: (str) a path to save the response at.



3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

**Auth object:**

In [62]:
auth = gather.get_auth(auth_file='auth.txt') # get authentication

**Archive data DataFrame:**

In [63]:
archive_original = pd.read_csv('twitter-archive-enhanced.csv', 
                               dtype={'tweet_id': str})
dump_lst = archive_original['tweet_id'] # to get list of id's as strings
archive_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   object 
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

***notice-loaded to json instead, relevant attributes only (rt and favorite counts):***

`gather.dump_tweets_data(id_lst=dump_lst, auth=auth)` **\*takes about 30 min, file already exists**

In [64]:
help(gather.dump_tweets_data) # doc for the above function

Help on function dump_tweets_data in module gather:

dump_tweets_data(id_lst, auth)
    Dump tweets' id, retweet count and favorite count into json file.
    
    :param id_lst: (list[str]) a list containing tweet id's.
    :param auth: (tweepy.auth.OAuth1UserHandler) an auth object.



**Tweets' counts DataFrame:**

In [65]:
tweet_cnts = pd.read_json('tweets_data.json', dtype={'id': str})
tweet_cnts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2356 non-null   object 
 1   favorite_count  2326 non-null   float64
 2   retweet_count   2326 non-null   float64
dtypes: float64(2), object(1)
memory usage: 55.3+ KB


**image predictions DataFrame:** 

In [66]:
img_preds = pd.read_csv('image_predictions.tsv', sep='\t', 
                        dtype={'tweet_id': str})
img_preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   object 
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


## Assessing Data
(Each dataframe has been assessed seperately below)

## **`archive_original` issues:**
### Quality issues
- *completeness*
    - missing `expanded_url` data. Detected visually - `archive_original.iloc[313]`
    - beacause we would like to clearly see if the relevant id is an original tweet, a boolean column
    `is_original` should be included.
- *validity* - 
    - inappropriate data type for columns `in_reply_to_status_id`,                   
    `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`: float -> object
    - inappropriate data type for columns `timestamp`, `retweeted_status_timestamp`: object -> datetime. both issues detected by using programmatic detection- `archive_original.info()`
    - invalid denominators in `rating_denominators`: all should be 10
    - duplicated links within the same cell in `expanded_url`. Detected visually - `archive_original['expanded_urls'].iloc[98]`
- *consistency* - 
    - name 'a' appears 55 times in the `name` column, probably representing unknown names, while there are defined None's in the column. Used visual, then programmatic detection-`archive_original['name'].value_counts()`
    - same values in `'doggo', 'floofer', 'pupper', 'puppo'` are counted as unique, probably because extra whitespace characters. Detected programmaticaly - `archive_original[['doggo', 'floofer', 'pupper', 'puppo']].value_counts()`
    
### Tidiness issues
- the 4 leftmost columns describe the same data, should be melted into one categorical column.
- `in_reply_to_status_id`, `in_reply_to_user_id` would be better reindexed to the left, as they consist mostly of NaN's and not really significant to the overall dataset

In [67]:
archive_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   object 
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [68]:
archive_original.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


## **`img_preds` issues:**
Used both programmatical and visual assessments, did not observe any issues. 

In [69]:
img_preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   object 
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


In [70]:
img_preds.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


## **`tweet_cnts` issues:**

### Tidiness issues
- dataframe would be better joined to archive_original, as it consists of general tweets data.

### Quality issues
- *validity*
    - count columns should be converted to int to avoid confusion.

In [71]:
tweet_cnts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2356 non-null   object 
 1   favorite_count  2326 non-null   float64
 2   retweet_count   2326 non-null   float64
dtypes: float64(2), object(1)
memory usage: 55.3+ KB


In [72]:
tweet_cnts.head()

,id,favorite_count,retweet_count
0,892420643555336193,32991.0,6913.0
1,892177421306343426,28530.0,5208.0
2,891815181378084864,21448.0,3435.0
3,891689557279858688,35998.0,7122.0
4,891327558926688256,34424.0,7644.0


## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
# Make copies of original pieces of data


### Issue #1:

#### Define:

#### Code

0    nan
1      4
dtype: object

#### Test

### Issue #2:

#### Define

#### Code

#### Test

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization